<a href="https://colab.research.google.com/github/jyotidabass/Chatbot/blob/main/Simple_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The code creates a SQLite database called "candidates.db" and a candidates table with columns for name, skills, budget, full_time, github_username, and experience_at_big_tech. It then inserts three sample candidate records into the table using the INSERT INTO statement. Finally, the connection to the database is closed.**

In [ ]:
import sqlite3

conn = sqlite3.connect("candidates.db")
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS candidates (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                skills TEXT,
                budget INTEGER,
                full_time INTEGER,
                github_username TEXT,
                experience_at_big_tech INTEGER
            )''')

c.execute("INSERT INTO candidates (name, skills, budget, full_time, github_username, experience_at_big_tech) VALUES (?, ?, ?, ?, ?, ?)",
          ("John Doe", "Python, JavaScript", 50000, 1, "johndoe", 1))
c.execute("INSERT INTO candidates (name, skills, budget, full_time, github_username, experience_at_big_tech) VALUES (?, ?, ?, ?, ?, ?)",
          ("Jane Doe", "React, Node.js", 30000, 0, "janedoe", 0))
c.execute("INSERT INTO candidates (name, skills, budget, full_time, github_username, experience_at_big_tech) VALUES (?, ?, ?, ?, ?, ?)",
          ("Bob Smith", "AWS, Java", 70000, 1, "bobsmith", 1))

conn.commit()
conn.close()

**This code imports necessary libraries and functions for text processing, connects to a SQLite database, and queries the database based on user input. It allows users to search for candidates with specific skills, budgets, full-time status, and experience at big tech companies. The process_query() function tokenizes, filters, and maps user input to corresponding categories, while the get_candidates() function constructs SQL queries based on the user's selections. The code runs in a loop, prompting the user for a new query after displaying the results.**

In [ ]:

import sqlite3
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import re

def process_query(query):
    # Tokenize the query
    tokens = word_tokenize(query)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Create a frequency distribution of the tokens
    fdist = FreqDist(lemmatized_tokens)

    # Map the tokens to their corresponding categories (skills, budget, etc.)
    categories = defaultdict(int)
    for token, freq in fdist.items():
        if re.match(r'\d+', token):
            categories['budget'] += freq
        elif token in ('full-time', 'part-time'):
            categories['full_time'] += freq
        elif token in ('python', 'react', 'aws'):
            categories['skills'] += freq
        elif token in ('big tech', 'google', 'facebook', 'amazon', 'microsoft'):
            categories['experience_at_big_tech'] += freq
        # Removed the 'else' block to avoid the 'other' category
        # else:
        #     categories['other'] += freq

    # Return the processed categories
    return categories

def get_candidates(skills=None, budget=None, full_time=None, github_username=None, experience_at_big_tech=None):
    # Set default values for all parameters to handle cases when they are not provided
    query_str = """
                SELECT * FROM candidates
                WHERE 1=1
                """
    params = ()

    if skills:
        query_str += " AND skills LIKE ?" # Use a placeholder for the LIKE parameter
        params += (f'%{skills}%',) # Pass the formatted skill string as a parameter
    if budget:
        query_str += " AND budget >= ?"
        params += (budget,)
    if full_time:
        query_str += " AND full_time = ?"
        params += (full_time,)
    if github_username:
        query_str += " AND github_username = ?"
        params += (github_username,)
    if experience_at_big_tech:
        query_str += " AND experience_at_big_tech = ?"
        params += (experience_at_big_tech,)

    return query_str, params

if __name__ == '__main__':
    # Connect to the SQLite database
    conn = sqlite3.connect('candidates.db')
    c = conn.cursor()

    # Prompt the user for the query
    while True:
        query = input("What are you looking for? ")
        categories = process_query(query)

        if not categories:
            print("Sorry, I need more information to assist you.")
        else:
            query_str, params = get_candidates(**categories) # This should work now as 'other' is not in categories and missing arguments are handled
            c.execute(query_str, params)
            candidates = c.fetchall()
            if candidates:
                print("Here are some candidates:")
                for candidate in candidates:
                    print(f"Name: {candidate[1]}, Skills: {candidate[2]}, Budget: {candidate[3]}, Full-Time: {candidate[4]}, GitHub Username: {candidate[5]}, Experience at Big Tech: {candidate[6]}")
            else:
                print("No candidates found.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Sorry, I need more information to assist you.
No candidates found.
Sorry, I need more information to assist you.
Sorry, I need more information to assist you.
Here are some candidates:
Name: John Doe, Skills: Python, JavaScript, Budget: 50000, Full-Time: 1, GitHub Username: johndoe, Experience at Big Tech: 1
Name: Bob Smith, Skills: AWS, Java, Budget: 70000, Full-Time: 1, GitHub Username: bobsmith, Experience at Big Tech: 1
No candidates found.
Sorry, I need more information to assist you.
Sorry, I need more information to assist you.
Here are some candidates:
Name: John Doe, Skills: Python, JavaScript, Budget: 50000, Full-Time: 1, GitHub Username: johndoe, Experience at Big Tech: 1
Name: Jane Doe, Skills: React, Node.js, Budget: 30000, Full-Time: 0, GitHub Username: janedoe, Experience at Big Tech: 0
Name: Bob Smith, Skills: AWS, Java, Budget: 70000, Full-Time: 1, GitHub Username: bobsmith, Experience at Big Tech: 1
Here are some candidates:
Name: John Doe, Skills: Python, JavaScript,